In [ ]:
from toolkit.common.constants import *
from toolkit.lapsim.multi_sim import MultiSim
from toolkit.cars.car_configuration import Car
from toolkit.lap.gps_importer import *
from toolkit.las_solvers import Octahedral_LAS, Multi_Layer_LAS, LAS
from toolkit.steady_state_solver import LS_Solver, Parachute

In [ ]:
tracks = []
# tracks = []
v_average = 15
mu_corr = 0.65 # mu correction factor, a value of 1 means the road is sandpaper and the actual value should be something lower but im kinda just setting this value to overfit atm
target = 0.001
solver = Parachute()
las = Octahedral_LAS(solver=solver)

In [ ]:
# here we make a function that generates a car with a given roll stiffness distribution and chassis roll stiffness
# the first and second arguments take the values of the first and second sweep variables that you give to run_sim
def gen_car_lltd(cop, cg):
    total_roll_stiffness = 700 * FTLB_TO_NM
    car = Car(front_axle_weight=cg)
    rsd = 0.5
    car.k_f = total_roll_stiffness * rsd
    car.k_r = total_roll_stiffness * (1-rsd)
    car.k_c = 14000 * FTLB_TO_NM
    car.front_axle_downforce = cop
    car.description = f"CoP:{cop:.2f} CG:{cg:.2f}"
    return car
# do a sweep of roll stiffness distribution and chassis roll stiffness
cop_range = np.linspace(0.35, 0.55, 10)
cg_r = np.geomspace(0.35, 0.55, 10)
sim = MultiSim(tracks, gen_car_lltd, cop_range, cg_r, f"CoP (% Front)", f"CG (% Front)")

In [ ]:
sim.run_sim(las=las)
sim.plot_LAS_corners()
sim.plot_LLTD()

In [ ]:
# plot velocity vs limit residual torque for each car
import plotly.graph_objects as go
fig = go.Figure()
for las, car in zip(sim.new_las, sim.new_cars):
    fig.add_trace(go.Scatter(x=las.vels, y=las.aymax[:, 2]*car.izz, name=f"CoP:{car.front_axle_downforce:.2f} CG:{car.front_axle_weight:.2f}"))
fig.update_layout(title="Velocity vs Limit Residual Torque", xaxis_title="Velocity (m/s)", yaxis_title="Limit Residual Torque (Nm)")
fig.show()